<a href="https://colab.research.google.com/github/curtcorum/mri_8ch_simulator_corrected_k_scaling_py/blob/main/3DRadialSimulation_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the updated notebook.

I have modified the `MRISimulator` to accept the `(1, Np, Nv, Ns, 3)` trajectory format. The simulation logic flattens this 5D array for vectorized calculation on the GPU/CPU and restores the shape (with the channel dimension appended) for the output.

For testing, I generate a synthetic "Koosh ball" style radial trajectory fitting your dimensions and display the resulting complex raw k-space data for the first few points.

### Colab-Ready Notebook

In [1]:
# @title 3D Radial Simulation (5D Input Format)
# @markdown Run this cell to simulate with input dimensions (1, Np, Nv, Ns, 3).

import sys
import subprocess
import numpy as np

# 1. Environment Setup
try:
    import cupy as cp
    HAS_CUPY = True
    print("CuPy is already installed.")
except ImportError:
    print("Checking for GPU...")
    try:
        subprocess.check_output('nvidia-smi')
        print("GPU detected. Installing CuPy...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'cupy-cuda12x'])
        import cupy as cp
        HAS_CUPY = True
        print("CuPy installed.")
    except Exception:
        print("No GPU. Using NumPy.")
        import numpy as cp
        HAS_CUPY = False

# --- CLASS DEFINITIONS ---

def Rz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

def point_array(N=[3,3,3]):
    """Generates integer offsets centered at 0."""
    Nx, Ny, Nz = N
    Cx, Cy, Cz = (Nx-1)/2, (Ny-1)/2, (Nz-1)/2
    x = (np.arange(Nx)-Cx)
    y = (np.arange(Ny)-Cy)
    z = (np.arange(Nz)-Cz)
    X,Y,Z = np.meshgrid(x,y,z, indexing='ij')
    return np.stack((X,Y,Z), axis=-1)

class CubeObject:
    def __init__(self, gain=1.0, kmax=1.0, L=[.5,.5,.5], D=[0,0,0], R=np.eye(3)):
        self.gain, self.kmax, self.L, self.D, self.R = gain, kmax, L, D, R

    def generate(self, kspace):
        xp = cp if (HAS_CUPY and isinstance(kspace, cp.ndarray)) else np
        k_flat = kspace.reshape(-1, 3)
        L = xp.asarray(self.L); D = xp.asarray(self.D); R = xp.asarray(self.R)

        k_rot = k_flat @ R
        D_rot = D @ R

        # Sinc Arguments
        argx = 0.5 * self.kmax * k_rot[:,0] * L[0]
        argy = 0.5 * self.kmax * k_rot[:,1] * L[1]
        argz = 0.5 * self.kmax * k_rot[:,2] * L[2]

        ph = xp.sum(0.5 * self.kmax * k_rot * D_rot, axis=1)

        sig = (self.gain * L[0] * xp.sinc(argx)) * \
              (L[1] * xp.sinc(argy)) * \
              (L[2] * xp.sinc(argz)) * \
              xp.exp(1j * 2 * np.pi * ph)
        return sig.reshape(kspace.shape[:-1])

class GaussianObject:
    def __init__(self, gain=1.0, kmax=1.0, L=[.5,.5,.5], D=[0,0,0]):
        self.gain, self.kmax, self.L, self.D = gain, kmax, L, D

    def generate(self, kspace):
        xp = cp if (HAS_CUPY and isinstance(kspace, cp.ndarray)) else np
        k_flat = kspace.reshape(-1, 3)
        L = xp.asarray(self.L); D = xp.asarray(self.D)
        kx = k_flat[:,0]*L[0]; ky = k_flat[:,1]*L[1]; kz = k_flat[:,2]*L[2]

        scale = 0.5 * self.kmax
        arg_sq = (scale**2) * (kx**2 + ky**2 + kz**2)
        ph = scale * xp.sum(k_flat * D, axis=1)

        return (self.gain * xp.exp(-np.pi*arg_sq) * xp.exp(1j*2*np.pi*ph)).reshape(kspace.shape[:-1])

class MRISimulator:
    def __init__(self, kspace_traj, matrix_size, nrcvrs=8):
        """
        kspace_traj: Array of shape (1, Np, Nv, Ns, 3)
        matrix_size: Scaling factor for object size in k-space (e.g. 64, 128)
        """
        self.kspaceRad = kspace_traj
        self.matrix_size = matrix_size
        self.nrcvrs = nrcvrs

        # Parse Input Dimensions
        shape = kspace_traj.shape
        if len(shape) != 5 or shape[-1] != 3:
             raise ValueError(f"Input trajectory must be (1, Np, Nv, Ns, 3). Got {shape}")

        self.Np = shape[1]
        self.Nv = shape[2]
        self.Ns = shape[3]

    def run_simulation(self):
        xp = cp if HAS_CUPY else np

        # 1. Grid Scaling
        dk = 2.0 / self.matrix_size

        # 2. Coil Kernels Setup
        pArray_int = point_array([5,5,5])
        pArray_phys = pArray_int * dk
        pArray_flat = pArray_phys.reshape(-1, 3)
        pArray_gpu = xp.asarray(pArray_flat)

        kernels = xp.zeros((pArray_flat.shape[0], self.nrcvrs), dtype=xp.complex64)

        # 8-Channel Ring
        radius = 0.5
        angles = np.linspace(0, 2*np.pi, self.nrcvrs, endpoint=False)
        coil_width = [1.0, 1.0, 1.0]

        for i in range(self.nrcvrs):
            loc = [radius * np.cos(angles[i]), radius * np.sin(angles[i]), 0.0]
            kernels[:, i] = GaussianObject(L=coil_width, kmax=self.matrix_size, D=loc).generate(pArray_gpu)
            kernels[:, i] /= xp.sum(xp.abs(kernels[:, i]))

        # 3. Trajectory Processing
        # Flatten the (1, Np, Nv, Ns, 3) input to (TotalPoints, 3)
        traj = xp.asarray(self.kspaceRad).reshape(-1, 3)

        # 4. Vectorized Kernel Convolution
        k_eval = traj[:, None, :] + pArray_gpu[None, :, :]
        k_eval_f = k_eval.reshape(-1, 3)

        # 5. Object Definition
        R_rot = xp.asarray(Rz(np.radians(10)))
        obj_main = CubeObject(gain=100.0, kmax=self.matrix_size, L=[.75,.75,.75], R=R_rot)
        obj_void = CubeObject(gain=-100.0, kmax=self.matrix_size, L=[.37,.37,.37], R=xp.asarray(Rz(np.radians(20))))

        sig = obj_main.generate(k_eval_f) + obj_void.generate(k_eval_f)

        # 6. Apply Channel Weights & Reshape
        sig_mtrx = sig.reshape(traj.shape[0], pArray_flat.shape[0])
        fid_flat = sig_mtrx @ kernels # (TotalPoints, nrcvrs)

        # Reshape to (1, Np, Nv, Ns, nrcvrs)
        return fid_flat.reshape(1, self.Np, self.Nv, self.Ns, self.nrcvrs)

# --- TESTING SECTION ---

# Parameters
Np = 64     # Readout points
Nv = 4096      # Views (Spokes)
Ns = 1       # Segments (Spheres)
n_ch = 4     # Channels
matrix_sz = 64

print(f"Generating Radial Trajectory: (1, {Np}, {Nv}, {Ns}, 3)")

# Generate a simple 3D radial trajectory
# Np points from -0.5 to 0.5 (scaled later by simulator logic)
#r = np.linspace(-1.0, 1.0, Np)
r = np.linspace(0.0, 1.0, Np)


# Generate random unit vectors for views
# (In a real scenario, this would be a proper view order)
phi = np.linspace(0, 2*np.pi, Nv * Ns)
theta = np.linspace(-np.pi, np.pi, Nv * Ns)

# Create trajectory array
traj_5d = np.zeros((1, Np, Nv, Ns, 3))

# Fill trajectory
count = 0
for s in range(Ns):
    for v in range(Nv):
        # Unit vector for this spoke
        u_vec = np.array([
            np.sin(theta[count]) * np.cos(phi[count]),
            np.sin(theta[count]) * np.sin(phi[count]),
            np.cos(theta[count])
        ])

        # Readout line: r * u_vec
        # r is (Np,), u_vec is (3,) -> (Np, 3)
        line = np.outer(r, u_vec)

        traj_5d[0, :, v, s, :] = line
        count += 1

# Initialize Simulator
sim = MRISimulator(traj_5d, matrix_size=matrix_sz, nrcvrs=n_ch)

print("Running Simulation...")
kdata_out = sim.run_simulation()

if HAS_CUPY:
    kdata_out = cp.asnumpy(kdata_out)

print(f"Simulation Complete. Output Shape: {kdata_out.shape}")
print("-" * 60)
print("Data Sample (First 5 points of 1st view, 1st segment):")
print(f"{'Point':<8} {'Coord (kx,ky,kz)':<25} {'Ch 1 (Real+Imag)':<20} {'Ch 2 (Real+Imag)':<20}")
print("-" * 60)

for i in range(5):
    coord = traj_5d[0, i, 0, 0, :]
    val_ch0 = kdata_out[0, i, 0, 0, 0]
    val_ch1 = kdata_out[0, i, 0, 0, 1]

    coord_str = f"[{coord[0]:.2f}, {coord[1]:.2f}, {coord[2]:.2f}]"
    ch0_str = f"{val_ch0.real:.2f}{val_ch0.imag:+.2f}j"
    ch1_str = f"{val_ch1.real:.2f}{val_ch1.imag:+.2f}j"

    print(f"{i:<8} {coord_str:<25} {ch0_str:<20} {ch1_str:<20}")

Checking for GPU...
No GPU. Using NumPy.
Generating Radial Trajectory: (1, 64, 4096, 1, 3)
Running Simulation...
Simulation Complete. Output Shape: (1, 64, 4096, 1, 4)
------------------------------------------------------------
Data Sample (First 5 points of 1st view, 1st segment):
Point    Coord (kx,ky,kz)          Ch 1 (Real+Imag)     Ch 2 (Real+Imag)    
------------------------------------------------------------
0        [-0.00, -0.00, -0.00]     29.53+0.00j          29.53-0.00j         
1        [-0.00, -0.00, -0.02]     22.57+0.00j          22.57+0.00j         
2        [-0.00, -0.00, -0.03]     7.16+0.00j           7.16+0.00j          
3        [-0.00, -0.00, -0.05]     -5.31-0.00j          -5.31-0.00j         
4        [-0.00, -0.00, -0.06]     -7.60-0.00j          -7.60-0.00j         
